In [ ]:
import polars as pl
import json

In [ ]:
with open('../params.json', 'r') as file :
    params = json.load(file)

DATASET, VERSION = params['dataset'], params['version']
DATA_FOLD = params['data_folder']

INPUT_FOLDER = f'{DATA_FOLD}/{VERSION}/2.clean_data/{DATASET}/temporal/'

In [ ]:
df = pl.read_parquet(INPUT_FOLDER + 'all_features_with_delta.parquet')

In [ ]:
cols_order = [
    # ── Identification & horodatage ───────────────────────────────
    "encounterId",          # identifiant unique du séjour
    "delta_hour",           # temps relatif (en heure))

    # ── Caractéristiques patient « lentes » ────────────────────────────
    "poids_suivi",          # poids
    "neuro_status",

    # ── Signes vitaux à haute fréquence ───────────────────────────────
    "heart_rate",
    "spo2",
    "pam_invasive", "pas_invasive", "pad_invasive",
    "pam_non_invasive", "pas_non_invasive", "pad_non_invasive",
    "fr",                   # fréquence respiratoire
    "temp",
    

    # ── Paramètres / dispositifs respiratoires ────────────────────────
    "mode_vent",
    "fio2",
    "o2_flow",
    "pep",                  # PEEP
    "tidal_volume",
    "pplat",
    "admin_o2",             # O₂ spontané (lunettes, masque…)
    "endotracheal_tube",
    "tracheo",
    "ecmo_type",
    "installation",         # binaire : décubitus ventrale ou non

    # ── Supports hémodynamiques & épuration ───────────────────────────
    "nad_dose_poids",       # noradrénaline
    "dobu_dose_poids",      # dobutamine
    "db_sang_cvvhf",        # débit CVVHDF
    "db_sang_hdi",          # débit hémodialyse intermittente

    # ── Balance entrées / sorties ─────────────────────────────────────
    "iv_input",
    "urine_output",

    # ── Biologie courante / POC ───────────────────────────────────────
    "glyc_cap",
    "lactate",
    "creat",
    "bili_tot",
    "hemoglobine",
    "leucocytes",
    "num_plq",
    "tp",

    # ── Produits sanguins transfusés ──────────────────────────────────
    "cgr",                  # concentrés globulaires
    "pfc",                  # plasma frais congelé
    "plq"                   # plaquettes
]


In [ ]:
import polars as pl

# ------------------------------------------------------------------
# 1.  Sélection des séjours qui possèdent au moins une valeur valide
# ------------------------------------------------------------------
key_features = ["heart_rate", "spo2"]
valid_mask = (pl.col("heart_rate").is_not_null()) | (pl.col("spo2").is_not_null())

df_filtered = (
    df
    .group_by("encounterId")
    .agg(
        [valid_mask.any().alias("has_valid")]  # True si au moins une mesure non nulle
    )
    .filter(pl.col("has_valid"))
    .select("encounterId")
    .join(df, on="encounterId")               # remet toutes les lignes du séjour
)

# ------------------------------------------------------------------
# 2.  Calcul de t0  (premier delta_hour valide) par séjour
# ------------------------------------------------------------------
df_shifted = (
    df_filtered
    # t0 = min(delta_hour) sur les lignes valides, réparti sur tout le séjour
    .with_columns(
        pl.when(valid_mask)
        .then(pl.col("delta_hour"))
        .otherwise(None)
        .min()
        .over("encounterId")
        .alias("t0")
    )
    # ------------------------------------------------------------------
    # 3.  Recentrage temporel
    # ------------------------------------------------------------------
    .with_columns(
        (pl.col("delta_hour") - pl.col("t0")).alias("delta_hour")
    )
    .drop("t0")  # facultatif : conserver ou non la colonne de référence
)

df_shifted.select(cols_order)


In [ ]:
df_shifted